In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
baseline = []  #packets sent in baseline
drl = []       #packets sent in DRL  
power_baseline = []
power_drl = []
Network_energy_drl=[]
Network_energy_baseline=[]

In [3]:
# import matplotlib.pyplot as plt
# import numpy as np

# X = np.array([i for i in range(1000)])
# y = np.array(baseline_power)
# z = np.array(drl_power)


# plt.plot(X, y, color='r', label='Baseline model')
# plt.plot(X, z, color='g', label='DRL based model')
  
# # Naming the x-axis, y-axis and the whole graph

# plt.xlabel("Episodes")
# plt.ylabel("Power Available")
# plt.title("Battery Drain w.r.to Baseline and DRL")
  
# # Adding legend, which helps us recognize the curve according to it's color
# plt.legend()
# plt.xlim([0, 1200]) 
# plt.ylim([0, 18000]) 
# plt.rcParams["figure.figsize"] = (14,7)
# # To load the display window
# plt.show()

In [4]:
class Wireless_Env_Baseline(Env):
    def __init__(self):
        
        #Packet Generation Rates calculated from Auto-Regression Model
        self.pgr = [1, 1, 1, 1, 1, 1, 1, 1]
        
        #These are packet queues for each sensor
        self.pulse = []
        self.heart_rate = []
        self.temperature = []
        self.diabetes = []
        self.spo2 = []
        self.ibi = []
        self.eda = []
        self.acc = []
      
        
        #Total initial power
        self.power = 4        
        
        
        #Step count for each episode. It can be changed as per requirement.
        self.count = 1000
        
        #self.sensors = {0 : "Pulse", 1 : "Heart Rate", 2 : "Temprature", 3 : "Diabetes",  : "SpO2", 5 : "EDA", 6 : "IBI", 7 : "ACC"}
                
        # Actions we can take ---> we can select any of 8 sensors for packet transmission.
        self.action_space = Discrete(8)
        
        
        #initial timestamp (Optional---> I have used it for testing purpose.)
        self.time_stamp = 1
        
        #Set the network length(Will update the battery power,,, We can update here the minimum battery sensor)
        self.life = 1000        
        
        #Initial number of packets in queues
        self.state = 0        
        
        
        #This is observation space..will take any random value( Just for model shake...No use in our case.)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
    
    
    #calculating reward based on our reward function.
    def get_reward(self, packet_gen_prob, self_queue, others_queue):
        #print("In Reward")
        reward_ = ((len(self_queue) * packet_gen_prob) + 1) / (others_queue - len(self_queue) + 1) + .25
        return reward_
     
        
    #This is crucial function. It defines how the state is changing after each selection. Every update is happening here.    
    def state_change(self): 
        
       
        
        #In baseline...adding 1 packet into each sensor queue as each sensor is sensing 1 packet per step.
        self.ibi.append(1)
        self.eda.append(1)        
        self.diabetes.append(1)        
        self.heart_rate.append(1)
        self.acc.append(1)       
        self.temperature.append(1)        
        self.pulse.append(1)       
        self.spo2.append(1)
            
        
    
    def step(self, action):       
        
        #for testing.
        self.count -= 1
        
        #Power equation used here. Can be changed as per requirement.
        self.power -= (16.31 * 10e-9 * 256 + 1.97 * 10e-9 * 100 * 256)
        
        #Testing
        #print("Selected sensor is --> ", action)
        
        #Changing the state.
        self.state_change()
        
        total_pending_packets = len(self.pulse) + len(self.heart_rate) + len(self.temperature) + len(self.diabetes) + len(self.spo2) + len(self.ibi) + len(self.eda) + len(self.acc) 
        
        #print("total pending packets -----> ", total_pending_packets)
        
        
        #In this if else ladder, we are updating based on selected node.
        
        if(self.count > 0):
            baseline.append(total_pending_packets)
#             drl.append(len(self.pulse) + len(self.heart_rate) + len(self.temperature) + len(self.diabetes))
        
        if(self.power > 0):
            power_baseline.append(self.power)
        if(self.power <= 0):
            power_baseline.append(0)
        Network_energy_baseline.append(self.power)
        if action == 0:
            #print(" Sensor Queue size is ", len(self.pulse))
            if(len(self.pulse) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[0], self.pulse, total_pending_packets)
                if(len(self.pulse) != 0):
                    self.pulse.pop()
        
        elif action == 1:
            #print(" Sensor Queue size is ", len(self.heart_rate))
            if(len(self.heart_rate) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[1], self.heart_rate, total_pending_packets)
                if len(self.heart_rate) != 0:
                    self.heart_rate.pop()
        
        elif action == 2:
            #print(" Sensor Queue size is ", len(self.temperature))
            if len(self.temperature) == 0:
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[2], self.temperature, total_pending_packets)        
                if len(self.temperature) != 0:
                    self.temperature.pop()
            
        elif action == 3:
            #print(" Sensor Queue size is ", len(self.diabetes ) )
            if(len(self.diabetes) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[3], self.diabetes, total_pending_packets)
                if len(self.diabetes) != 0:
                    self.diabetes.pop()
        
        elif action == 4:
            #print(" Sensor Queue size is ", len(self.spo2))
            if(len(self.spo2) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[4], self.spo2, total_pending_packets)        
                if len(self.spo2) != 0:
                    self.spo2.pop()
        elif action == 5:
            #print(" Sensor Queue size is ",len(self.eda))
            if(len(self.eda) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[5], self.eda, total_pending_packets)        
                if len(self.eda) != 0:
                    self.eda.pop()
        elif action == 6:
            #print(" Sensor Queue size is ", len(self.ibi))
            if(len(self.ibi) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[6], self.ibi, total_pending_packets)        
                if len(self.ibi) != 0:
                    self.ibi.pop()
        else:
            #print(" Sensor Queue size is ",len(self.acc))
            if(len(self.acc) == 0):
                reward = 0
            else:
                
                reward = self.get_reward(self.pgr[7], self.acc, total_pending_packets)        
                if len(self.acc) != 0:
                    self.acc.pop()  
        
        
        #Reducing life by 1, here we can also use battery power. 
        self.life -= 1    
        
       
        
        if self.life <= 0: 
            done = True
        else:
            done = False   
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        #For Visualization Purpose
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 0
        # Reset shower time
        self.life = 1000
        
        #Reset all packet queues
        self.pulse = []
        self.heart_rate = []
        self.temperature = []
        self.diabetes = []
        self.spo2 = []
        self.ibi = []
        self.eda = []
        self.acc = []
        
        return self.state
    
    

In [5]:

#Here also, all parameters are same as Baseline Model apart from PGR.
class Wireless_Env_DRL(Env):
    def __init__(self):
        
        #Packet Generation Rates calculated from Auto-Regression Model
        self.pgr = [0.87, 0.069, 0.001, 0.631, 0.03, 0.974, 0.938, 0.159]
        
        self.pulse = []
        self.heart_rate = []
        self.temperature = []
        self.diabetes = []
        self.spo2 = []
        self.ibi = []
        self.eda = []
        self.acc = []
        
        self.power = [.5, .5, .5, .5, .5, .5, .5, .5]
        self.count = 1001
        #self.flag = 1
        #self.sensors = {0 : "Pulse", 1 : "Heart Rate", 2 : "Temprature", 3 : "Diabetes",  : "SpO2", 5 : "EDA", 6 : "IBI", 7 : "ACC"}
                
        # Actions we can take, down, stay, up
        self.action_space = Discrete(8)
        
        self.time_stamp = 1
        
        #Set the network length(Will update the battery power,,, We can update here the minimum battery sensor)
        self.life = 1000        
        
        #Initial number of packets in queues
        self.state = 0        
        
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        
    def get_reward(self, packet_gen_prob, self_queue, others_queue):
        #print("In Reward")
        reward_ = (((len(self_queue) * packet_gen_prob) + 1) / (others_queue - len(self_queue) + 1) + .25)
        return reward_
        
    def state_change(self): 
        
        ts = self.time_stamp
        
        self.time_stamp += 1
        
        self.ibi.append(1)
        self.eda.append(1) 
        if ts % 2 == 0:
           self.diabetes.append(1)        
        
        if ts % 14 == 0:
            self.heart_rate.append(1)
        
        if ts % 6 == 0:
            self.acc.append(1)
        
        if ts % 1000 == 0:
            self.temperature.append(1)
            
        if ts % 11 == 0:
            self.pulse.append(1)            
            
        if ts % 30 == 0:
            self.spo2.append(1)
            
        
    
    def step(self, action):       
        
        self.count -= 1
        
        #Power function. But we will reduce power only when sensor is selected.
        p = (16.31 * 10e-9 * 256 + 1.97 * 10e-9 * 100 * 256)
        
        
        #print("Selected sensor is --> ", action)
        
        self.state_change()
        
        total_pending_packets = len(self.pulse) + len(self.heart_rate) + len(self.temperature) + len(self.diabetes) + len(self.spo2) + len(self.ibi) + len(self.eda) + len(self.acc) 
        
        #print("total pending packets -----> ", total_pending_packets)
        
        if(self.count > 0):
            
            drl.append(total_pending_packets)
            
        if(sum(self.power) > 0):
            power_drl.append(sum(self.power))
        
        Network_energy_drl.extend(self.power)
        if action == 0:
            #print(" Sensor Queue size is ", len(self.pulse))
            if(len(self.pulse) == 0):
                reward = 0
            else:
                if(self.power[0] - p > 0): #here I am reducing power if this sensor is selected. same for others as well.
                    self.power[0] -= p
                    #print(power)
                reward = self.get_reward(self.pgr[0], self.pulse, total_pending_packets)
                if(len(self.pulse) != 0):
                    self.pulse.pop()
        
        elif action == 1:
            #print(" Sensor Queue size is ", len(self.heart_rate))
            if(len(self.heart_rate) == 0):
                reward = 0
            else:
                if(self.power[1] - p > 0):
                    self.power[1] -= p
                reward = self.get_reward(self.pgr[1], self.heart_rate, total_pending_packets)
                if len(self.heart_rate) != 0:
                    self.heart_rate.pop()
        
        elif action == 2:
            #print(" Sensor Queue size is ", len(self.temperature))
            if len(self.temperature) == 0:
                reward = 0
            else:
                if(self.power[2] - p > 0):
                    self.power[2] -= p
                reward = self.get_reward(self.pgr[2], self.temperature, total_pending_packets)        
                if len(self.temperature) != 0:
                    self.temperature.pop()
            
        elif action == 3:
            #print(" Sensor Queue size is ", len(self.diabetes ) )
            if(len(self.diabetes) == 0):
                reward = 0
            else:
                if(self.power[3] - p > 0):
                    self.power[3] -= p
                reward = self.get_reward(self.pgr[3], self.diabetes, total_pending_packets)
                if len(self.diabetes) != 0:
                    self.diabetes.pop()
        
        elif action == 4:
            #print(" Sensor Queue size is ", len(self.spo2))
            if(len(self.spo2) == 0):
                reward = 0
            else:
                if(self.power[4] - p > 0):
                    self.power[4] -= p
                reward = self.get_reward(self.pgr[4], self.spo2, total_pending_packets)        
                if len(self.spo2) != 0:
                    self.spo2.pop()
        elif action == 5:
            #print(" Sensor Queue size is ",len(self.eda))
            if(len(self.eda) == 0):
                reward = 0
            else:
                if(self.power[5] - p > 0):
                    self.power[5] -= p
                reward = self.get_reward(self.pgr[5], self.eda, total_pending_packets)        
                if len(self.eda) != 0:
                    self.eda.pop()
        elif action == 6:
            #print(" Sensor Queue size is ", len(self.ibi))
            if(len(self.ibi) == 0):
                reward = 0
            else:
                if(self.power[6] - p > 0):
                    self.power[6] -= p
                reward = self.get_reward(self.pgr[6], self.ibi, total_pending_packets)        
                if len(self.ibi) != 0:
                    self.ibi.pop()
        else:
            #print(" Sensor Queue size is ",len(self.acc))
            if(len(self.acc) == 0):
                reward = 0
            else:
                if(self.power[7] - p > 0):
                    self.power[7] -= p
                reward = self.get_reward(self.pgr[7], self.acc, total_pending_packets)        
                if len(self.acc) != 0:
                    self.acc.pop()  
        
        
        
        self.life -= 1    
            
        if self.life <= 0: 
            done = True
        else:
            done = False   
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        #For Visualization Purpose
        pass
    
    #This step resets the parameters after each 1000 steps. For training purpose. can be updated for battery life as well.
    def reset(self):
        # Reset shower temperature
        self.state = 0
        # Reset shower time
        self.life = 1000
        
        #Reset all packet queues
        self.pulse = []
        self.heart_rate = []
        self.temperature = []
        self.diabetes = []
        self.spo2 = []
        self.ibi = []
        self.eda = []
        self.acc = []
        
        return self.state
    


In [ ]:
env_drl = Wireless_Env_DRL() 

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
env_bsl = Wireless_Env_Baseline()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
env_bsl.observation_space.sample()

array([96.919495], dtype=float32)

In [ ]:
env_drl.observation_space.sample()

array([71.05866], dtype=float32)

In [ ]:
 episodes = 10
 sco_drl = []
 for episode in range(1, episodes+1):
     state = env_drl.reset()
     done = False
     score = 0 
    
     while not done:
         #env.render()
         action = env_drl.action_space.sample()
         n_state, reward, done, info = env_drl.step(action)
         score+=reward
     print('Episode:{} Reward:{}'.format(episode, score))
     sco_drl.append(score)

Episode:1 Reward:349.54727344129896
Episode:2 Reward:360.64394253198634
Episode:3 Reward:355.8371876822331
Episode:4 Reward:357.5610175856082
Episode:5 Reward:347.85794665782515
Episode:6 Reward:340.2428187488734
Episode:7 Reward:327.29616765046245
Episode:8 Reward:324.7810367100896
Episode:9 Reward:343.3053840984981
Episode:10 Reward:352.57764045761473


In [ ]:
 episodes = 10
 sco_bsl=[]
 for episode in range(1, episodes+1):
     state = env_bsl.reset()
     done = False
     score = 0 
    
     while not done:
         #env.render()
         action = env_bsl.action_space.sample()
         n_state, reward, done, info = env_bsl.step(action)
         score+=reward
     print('Episode:{} Reward:{}'.format(episode, score))
     sco_drl.append(score)    

Episode:1 Reward:393.9158701954011
Episode:2 Reward:393.89496487273715
Episode:3 Reward:393.926887055047
Episode:4 Reward:394.05180377123577
Episode:5 Reward:393.85808160420646
Episode:6 Reward:393.7032283836826
Episode:7 Reward:393.93482136707706
Episode:8 Reward:393.7494541594191
Episode:9 Reward:393.9048926552631
Episode:10 Reward:393.70660544762353


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#A = np.array(sco_drl)

In [ ]:
#pd.DataFrame(A).to_csv('/content/drive/MyDrive/Colab Notebooks/reward_drl_e4.csv')

In [ ]:
import numpy as np
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
model = keras.Sequential()

In [ ]:
states = env_bsl.observation_space.shape
actions = env_bsl.action_space.n

In [ ]:
states = env_drl.observation_space.shape
actions = env_drl.action_space.n

In [ ]:
env_drl.observation_space

Box(0.0, 100.0, (1,), float32)

In [ ]:
env_bsl.observation_space

Box(0.0, 100.0, (1,), float32)

In [ ]:
actions

8

In [ ]:
del model

In [ ]:
def build_model(states, actions):
    model = keras.Sequential()
    
    model.add(Dense(32, activation='relu', input_shape=states))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 32)                64        
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 8)                 520       
                                                                 
Total params: 2,696
Trainable params: 2,696
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!pip install keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=12000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
#Run DRL based model
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env_drl, nb_steps=12000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 12000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 29:06 - reward: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 109s 11ms/step - reward: 0.3726
10 episodes - episode_reward: 372.564 [351.724, 405.948] - loss: 0.111 - mae: 3.929 - mean_q: 4.553

Interval 2 (10000 steps performed)
 2000/10000 [=====>........................] - ETA: 1:15 - reward: 0.3973done, took 127.555 seconds


In [ ]:
#sscores = dqn.test(env_drl, nb_episodes = 300, visualize = False)

Testing for 300 episodes ...
Episode 1: reward: 257.677, steps: 1000
Episode 2: reward: 257.009, steps: 1000
Episode 3: reward: 257.679, steps: 1000
Episode 4: reward: 257.427, steps: 1000
Episode 5: reward: 257.747, steps: 1000
Episode 6: reward: 257.846, steps: 1000
Episode 7: reward: 257.656, steps: 1000
Episode 8: reward: 257.791, steps: 1000
Episode 9: reward: 257.668, steps: 1000
Episode 10: reward: 257.678, steps: 1000
Episode 11: reward: 257.741, steps: 1000
Episode 12: reward: 258.013, steps: 1000
Episode 13: reward: 257.066, steps: 1000
Episode 14: reward: 257.582, steps: 1000
Episode 15: reward: 257.763, steps: 1000
Episode 16: reward: 257.359, steps: 1000
Episode 17: reward: 257.766, steps: 1000
Episode 18: reward: 257.809, steps: 1000
Episode 19: reward: 257.664, steps: 1000
Episode 20: reward: 257.774, steps: 1000
Episode 21: reward: 258.003, steps: 1000
Episode 22: reward: 257.669, steps: 1000
Episode 23: reward: 257.577, steps: 1000
Episode 24: reward: 257.344, steps: 1

In [ ]:
len(Network_energy_drl)

96000

In [ ]:
my_array = np.array(Network_energy_drl)

In [ ]:
NTE=np.reshape(Network_energy_drl,(12000,8))

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(NTE)

In [ ]:
df

,0,1,2,3,4,5,6,7
0,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.500000,0.500000
1,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.500000,0.500000
2,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.500000,0.500000
3,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.499454,0.500000
4,0.500000,0.500000,0.5,0.500000,0.500000,0.500000,0.499454,0.500000
...,...,...,...,...,...,...,...,...
11995,0.000343,0.036930,0.5,0.000343,0.285393,0.000343,0.000343,0.000343
11996,0.000343,0.036930,0.5,0.000343,0.285393,0.000343,0.000343,0.000343
11997,0.000343,0.036930,0.5,0.000343,0.285393,0.000343,0.000343,0.000343
11998,0.000343,0.036930,0.5,0.000343,0.285393,0.000343,0.000343,0.000343


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Engergy_e4.csv')

In [ ]:
len(power_drl)

12000

In [ ]:
arti1 = np.array(power_drl)

In [ ]:
pd.DataFrame(arti1).to_csv('/content/drive/MyDrive/Colab Notebooks/total_energy_e4.csv')

In [ ]:
len(drl)

1000

In [ ]:
arti = np.array(drl)

In [ ]:
pd.DataFrame(arti).to_csv('/content/drive/MyDrive/Colab Notebooks/packet_e4.csv')

In [ ]:
#Run Baseline Model
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env_bsl, nb_steps=12000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 12000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 43:35 - reward: 0.5000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 103s 10ms/step - reward: 0.3927
10 episodes - episode_reward: 392.738 [391.052, 393.562] - loss: 0.219 - mae: 13.126 - mean_q: 15.279

Interval 2 (10000 steps performed)
 1996/10000 [====>.........................] - ETA: 1:20 - reward: 0.3926done, took 123.183 seconds


In [ ]:
#sscores = dqn.test(env_bsl, nb_episodes = 100, visualize = False)

Testing for 100 episodes ...
Episode 1: reward: 252.078, steps: 1000
Episode 2: reward: 252.078, steps: 1000
Episode 3: reward: 252.078, steps: 1000
Episode 4: reward: 252.078, steps: 1000
Episode 5: reward: 252.078, steps: 1000
Episode 6: reward: 252.078, steps: 1000
Episode 7: reward: 252.078, steps: 1000
Episode 8: reward: 252.078, steps: 1000
Episode 9: reward: 252.078, steps: 1000
Episode 10: reward: 252.078, steps: 1000
Episode 11: reward: 252.078, steps: 1000
Episode 12: reward: 252.078, steps: 1000
Episode 13: reward: 252.078, steps: 1000
Episode 14: reward: 252.078, steps: 1000
Episode 15: reward: 252.078, steps: 1000
Episode 16: reward: 252.078, steps: 1000
Episode 17: reward: 252.078, steps: 1000
Episode 18: reward: 252.078, steps: 1000
Episode 19: reward: 252.078, steps: 1000
Episode 20: reward: 252.078, steps: 1000
Episode 21: reward: 252.078, steps: 1000
Episode 22: reward: 252.078, steps: 1000
Episode 23: reward: 252.078, steps: 1000
Episode 24: reward: 252.078, steps: 1

In [ ]:
len(Network_energy_drl)
my_array = np.array(Network_energy_drl)
NTE=np.reshape(Network_energy_drl,(12000,8))

In [ ]:
len(Network_energy_baseline)

12000

In [ ]:
len(power_baseline)

12000

In [ ]:
manish = np.array(power_baseline)

In [ ]:
pd.DataFrame(manish).to_csv('/content/drive/MyDrive/Colab Notebooks/total_energy_baseline.csv')

In [ ]:
manish = np.array(Network_energy_drl)


In [ ]:
pd.DataFrame(manish).to_csv('/content/drive/MyDrive/Colab Notebooks/Energy_e4.csv')

In [ ]:
print(len(drl))
print(len(baseline))
baseline.append(baseline[-1])
print(len(baseline))


1000
999
1000


In [ ]:
print(power_drl)

[4.0, 3.9994539264, 3.9994539264, 3.9989078528, 3.9989078528, 3.9989078528, 3.9989078528, 3.9989078528, 3.9989078528, 3.9989078528, 3.9983617791999997, 3.9978157056, 3.997269632, 3.9967235584000003, 3.9961774848000005, 3.9961774848000005, 3.9956314112000006, 3.9950853376000004, 3.994539264, 3.994539264, 3.9939931904, 3.9934471168, 3.9934471168, 3.9934471168, 3.9929010432000003, 3.9923549696000005, 3.9923549696000005, 3.991808896, 3.9912628224000004, 3.9907167488000006, 3.990170675200001, 3.9896246016000005, 3.9890785280000007, 3.9885324544000005, 3.9879863808000007, 3.9879863808000007, 3.9874403072000004, 3.9868942336000006, 3.986348160000001, 3.985802086400001, 3.985802086400001, 3.985802086400001, 3.985256012800001, 3.9847099392000005, 3.9841638656000007, 3.983617792000001, 3.983071718400001, 3.9825256448000013, 3.9819795712000015, 3.981433497600001, 3.981433497600001, 3.981433497600001, 3.9808874240000014, 3.9803413504000007, 3.9803413504000007, 3.979795276800001, 3.979249203200001,

In [ ]:
#Taking every 1000th sample for plotting in DRL.
res = []
i = 0

while(i < 12000):
    res.append(power_drl[i])
    i += 1000
res.append(power_drl[-1])    


In [ ]:
print(res)


[4.0, 3.4910594048000236, 2.953722982400049, 2.4180247808000734, 1.87522762240008, 1.33079224320008, 0.816390912000081, 0.6170740480000815, 0.5744803072000819, 0.535163008000082, 0.5013064448000824, 0.4729106176000827, 0.4374158336000828]


In [ ]:
#Taking every 1000th sample for plotting in Baseline.

res1 = []
i = 0

while(i < 12000):
    res1.append(power_baseline[i])
    i += 1000
    
res1.append(power_baseline[-1])